
# ClassicVC model structure

Lyodos 著

Version 1.0.0 (2024-07-14)

このノートブックでは、ClassicVC の PyTorch モデルをインスタンス化する方法を示す。
あわせて各コンポーネントの解説を行う。
なお開発者はプログラミングや信号処理の教育を受けたことがなく、本業は生物系（しかも野外で動物さん達を観察する）なので、
用語や概念の解釈がしばしば的外れな点は容赦いただきたい。

## 総論

ClassicVC は連続潜在変数ベースで any-to-any 声質変換を行うことを目指して開発された。
基本的な設計は AutoVC の流れをくむエンコーダ・デコーダモデルである。

すなわちソース音声から content encoder で発話内容の埋め込みを取り出し、これを話者スタイルの時間次元を持たないベクトルと組み合わせ、
デコーダに投入する。デコーダはいわゆるニューラルボコーダの構造を踏襲しており、潜在空間から直接 24000 Hz 音声を復元できる。
訓練時は、発話内容と同じソース音声から得た話者スタイルを用いて自己再構成を行い、
推論時は別のターゲット話者から得た話者スタイルと合わせて、そのターゲットの声質を再現する。

また、最初期の研究例と現代の VC モデルとの違いとして、基本ピッチ（F0）をソース音声から推定し、その値をデコーダに教えることで、
音高変化を自由に制御することが可能になっている。
ClassicVC ではさらに、ソース音声の F0 を基準としてプラスマイナスいくつで音高をシフトさせるか、
あるいは発話内容とターゲット話者スタイルの組み合わせに基づき、ターゲットが話すであろう F0 を推定するかという、
音高のベースラインの選択が可能である。

これらの強力な調整機能は、いくつかの主要なコンポーネントを StyleTTS 2 (Li et al. 2023 https://arxiv.org/abs/2306.07691) 
から踏襲したことで可能となった。
モデルの[ソースコード](https://github.com/yl4579/StyleTTS2)をオープンソースで公開してくださっている著者の方々に、
あらめて謝意を表する。
ただしこのモデルは名前のとおり TTS (text to speech) 用であるから、VC タスクを実現するために細かな差し替えを独自に行った。
概要を以下に示す。

### コンポーネント概要図

![コンポーネント概要図](https://github.com/lyodos/classic-vc/raw/main/docs/ClassicVC_overview-en.png)

なお、元になった StyleTTS 2 の概要図も以下に示す。
TTS よりも VC の方が（phoneme duration を推定しなくていいため）かなり構造が簡単になることがお分かりだろう。
また StyleTTS 2 の核心部分である style diffusion 拡散モデルについては、訓練にも推論にも時間を使うため削除した。

![StyleTTS 2 overview 1](https://github.com/lyodos/classic-vc/raw/main/docs/styleTTS2_01.png)

![StyleTTS 2 overview 2](https://github.com/lyodos/classic-vc/raw/main/docs/styleTTS2_02.png)

![StyleTTS 2 overview 3](https://github.com/lyodos/classic-vc/raw/main/docs/styleTTS2_03.png)

見てのとおり StyleTTS 2 はモデル全体が巨大なため、訓練は two-stage で実施される。
ClassicVC は幸い、そこまで大きくないので VRAM 24 GB の GPU を 1 台使い、single-stage で訓練が可能である。

では、さっそくモデルの各コンポーネントをノートブック上で定義していこう。


----

## 準備

データを置くディレクトリと、使用する GPU を指定する。

プロジェクト（ClassicVC）全体で重量級のデータを共有するフォルダは、アクセスが高速なストレージに置く必要がある。
ClassicVC のリポジトリのクローン自体（および、このノートブック）は別の場所に配置してもいい。


In [ ]:
%%time
from pathlib import Path

# チェックポイントやログを保存する、機械学習関連のデータを置くルートディレクトリの指定

DATASET_ROOT_PATH = Path("/home/lyodos/study/dataset") # このフォルダ名はユーザーの実情に合わせて書き変えること

proj_path = DATASET_ROOT_PATH / "checkpoints" / "classic-vc"
proj_path.mkdir(parents = True, exist_ok = True)
print("Project directory:", str(proj_path))



モデルの評価に使う GPU を指定する。現在マルチ GPU には対応していない。


In [ ]:
import torch

if torch.cuda.device_count() >= 2:
    device = torch.device('cuda:1') # 複数ある場合、よわよわの GPU を評価用に使う（強いほうは訓練用に空ける）
elif torch.cuda.device_count() >= 1:
    device = torch.device('cuda:0')
else:
    device = torch.device('cpu')



以下、ネットワークの部品を定義していく。


----

## HarmoF0 pitch tracker の定義

このモデルは Wei et al (2022) https://doi.org/10.1109/ICME52920.2022.9858935 で提案された。

> HarmoF0: Logarithmic Scale Dilated Convolution For Pitch Estimation
> Weixing Wei, Peilin Li, Yi Yu, Wei Li

HarmoF0 公式実装および訓練済みの重みは https://github.com/WX-Wei/HarmoF0 において、MIT License で公開されている。
ClassicVC ではこれらを元ライセンスに準拠して転載しているが、ONNX への変換のためにコードを部分的に書き変えている（重みは転用可能）。

以下のコンポーネントにソース音声を投入して、時間フレーム単位の基本ピッチ（F0）、音量（energy）、アクティベーション（現在鳴っているのが人声か背景雑音か）を推定する。あわせて、推定過程で必要となる対数スペクトログラムも取り出す。

* 入力は 16000 Hz waveform で、ホップで 1/160 になるので 10 ms 解像度である。

* 入力は `torch.Size([n_batch, n_sample])` ただし `(1, 513)` （32.1 ms）以上の wave length がないとエラーになる。

### Wav2spec モジュール

実際の HarmoF0 ネットワーク（畳み込みネットワーク）への入力は、対数スペクトログラムの形で行われる。
ただし `torch.stft` や `librosa.stft` 等のコードを使うと、ONNX に変換するときに複素数に対応していないという理由でエラーが出る。

なので wav2spec の部分だけは Prem Seetharaman による "STFT/iSTFT in PyTorch" https://github.com/pseeth/pytorch-stft を採用し、
古典的な短時間フーリエ変換を畳み込みに置換したモジュール（このリポジトリの `conv_stft.py`）を使用する。
これにより、waveform を放り込んで F0 やスペクトログラムを取り出すお手軽機能が、可搬性の高い ONNX 形式で使えるようになる。

> なお、`conv_stft.py` のコードだけは元実装に従い、MIT ではなくBSD 3-Clause License でリリースする。
それぞれのソースファイル冒頭にライセンスが記載されているので、確認してほしい。


In [ ]:
import torch
import torchaudio
import sys
sys.path.append('../') # ClassicVC のリポジトリのルートをパスに入れて、model ディレクトリを探せるようにしている

from model.harmof0.pitch_tracker import BatchedPitchEnergyTracker

def pred_f0_len(length):
    return length // 160 + 1

harmof0_tracker = BatchedPitchEnergyTracker(
    checkpoint_path = "../model/harmof0/checkpoints/mdb-stem-synth.pth", # HarmoF0 作者による訓練済みの重みを再配布
    fmin = 27.5, # f0 として想定する最低周波数の Hz で、ピアノの最低音の A に相当する。
    sample_rate = 16000,
    hop_length = 160, # f0 を推定する間隔。160/16000 = 10 ms 
    frame_len = 1024, # sliding window を切り出す長さ
    frames_per_step = 1000, # 1 回の forward で投入する最大セグメント数
    high_threshold = 0.8, 
    low_threshold = 0.1, 
    freq_bins_in = 88*4,
    bins_per_octave_in = 48,
    bins_per_octave_out = 48,
    device = device,
    compile = False,
    dry_run = 10, 
)

# ちなみに初期化時点でネットワークの重みは freeze 済み


なお、コンポーネントが正常に動作していることを確かめるために適当な音声を突っ込んでみる。
以下のサンプルは、VCTK コーパス (The Centre for Speech Technology Research, University of Edinburgh) から Open Data Commons Attribution License (ODC-By) v1.0. に従い転載した。

> なお VCTK コーパスは ClassicVC の訓練には一切用いておらず（Notebook 02 も参照）、ここでは未知話者への適用（zero-shot）となる。


In [ ]:
# VCTK コーパスからのボイスサンプルを読み込んでみる
debug = True
if debug == True:  
    waveform, orig_sr = torchaudio.load('../wavs/p225_003.wav') # まずオリジナル周波数でロード
    wav16 = torchaudio.transforms.Resample(orig_freq = orig_sr, new_freq = 16000)(waveform).to(device)
    with torch.no_grad():
        %time freq_t, act_t, energy_t, spec = harmof0_tracker(wav16) # 初回だけ遅いので 2 回通して時間を測る
        %time freq_t, act_t, energy_t, spec = harmof0_tracker(wav16)
    print(freq_t.shape, act_t.shape, energy_t.shape, spec.shape)


import IPython.display as ipd

ipd.Audio(waveform, rate = orig_sr, normalize = False)



ネットワークの構造も調査しておく


In [ ]:
import torchinfo

with torch.no_grad():
    %time pitch_map = harmof0_tracker.single_tracker.net(spec)
print(pitch_map.shape)

torchinfo.summary(model = harmof0_tracker.single_tracker.net, input_size = spec.shape, depth = 4, 
                  col_names=["input_size", "output_size", "num_params"], device = device)




実際に声のピッチとエネルギーをプロットしてみる。


In [ ]:
plot_example = True

if plot_example:
    import matplotlib.pyplot as plt
    import matplotlib.cm as cm

    %matplotlib inline
    plt.rcParams['figure.figsize'] = (16.0, 6.0) # default 6.4, 4.8

    figure, axis = plt.subplots(1, 1)
    axis.set_title("F0 Feature")
    axis.grid(True)

    # x 軸（時間軸）のセット
    end_time = waveform.shape[-1] / orig_sr
    axis.set_ylim((-1.3, 1.3))
    time_axis = torch.linspace(start = 0, end = end_time, steps = waveform.shape[-1]) # steps は等差数列の要素数

    # 音声波形を第 1 軸にプロット
    ln1 = axis.plot(
        time_axis, 
        waveform.detach().squeeze().cpu().numpy(), 
        linewidth = 1, 
        color = 'gray', 
        label = 'Waveform', 
        alpha = 0.3,
    )

    axis2 = axis.twinx()
    axis2.set_ylim((0, 600))

    # HarmoF0 pitch sequence
    time_axis = torch.linspace(start = 0, end = end_time, steps = freq_t.shape[-1])
    ln4 = axis2.plot(
        time_axis, 
        freq_t.squeeze().detach().cpu().numpy(), 
        linewidth = 1, 
        linestyle = "dashed",
        label = 'Frequency (HarmoF0)', 
        color = "black",
    )

    # HarmoF0 activation sequence
    time_axis = torch.linspace(start = 0, end = end_time, steps = act_t.shape[-1])
    ln5 = axis.plot(
        time_axis, 
        act_t.squeeze().detach().cpu().numpy(),
        linewidth = 1, 
        label = "Activation (HarmoF0)", 
        color = cm.hsv(0.7),
    )
    
    # HarmoF0 energy sequence
    time_axis = torch.linspace(start = 0, end = end_time, steps = act_t.shape[-1])
    ln3 = axis.plot(
        time_axis, 
        energy_t.squeeze().detach().cpu().numpy() / 10, 
        linewidth = 1, 
        label = "Energy (HarmoF0) / 10", 
        color = cm.hsv(0.9),
    )

    # 合成
    lns = ln1 + ln4 + ln5 + ln3
    labels = [l.get_label() for l in lns] # 重複回避のためラベルを結合してから描画する
    axis.legend(lns, labels, loc = 'upper right')

    plt.show(block = False)


In [ ]:
from model.utils import plot_spectrogram_harmof0

plot_spectrogram_harmof0(
    spec.squeeze().cpu()[48:, :],
    f0 = freq_t.detach().cpu(), 
    act = act_t.detach().cpu(), 
    size = (12, 4.5),
    aspect = None,
    vmin = -50,
    vmax = 40,
    cmap = "inferno",
)
    


青い破線が activation である。概ね声がある場所で上側（1）に張り付いていることが分かるだろう。

ちなみにスペクトログラムの下側 48 bins を捨ててから図示しているが、これは低音側 1 オクターブ（27.5 Hz ～ 55 Hz）
に人声が存在しないだろうとの想定から、後工程のネットワークに明らかに不要な周波数帯を与えないことで軽量化する工夫である。


----

## (Acoustic) Style Encoder の初期化

$E_{a}(x)$ を作成する。構造は StyleTTS 2 のものを、入力の feature dimension を HarmoF0 に合うよう改変して採用している。
なので既存の重みは使用できず、自分で作る必要がある。

入力はスペクトログラムで、さらに `(batch, 1, dim_spec, n_frame >= 80)` の 4D テンソルでないと受けられない。
なので上で作った spec を unsqueeze(1) して入れることになる。

また HarmoF0 の低音側は 27.5 Hz だが、下の 1 オクターブ（48 bins）は人の声で使うことはなく無駄なので、
入力は 352 ではなく 352 - 48 = 304 にする。これでパラメータ数を 23,719,264 → 23,189,104  に削減できる。

出力は時間次元を持たない `(batch, 128)`。


> このモデル、そこそこの計算時間を要する。VRAM に余裕があれば Transformer ベースもしくは状態空間モデルベースで、
同等機能を持つものを開発して置き換えるとよいだろう。



In [ ]:
from dataclasses import dataclass
import typing
from omegaconf import OmegaConf

from model.StyleTTS2.models import StyleEncoder


@dataclass
class StyleEncoderConfig:
    dim_in: int = 304
    style_dim: int = 128
    max_conv_dim: int = 512

style_encoder_cfg = OmegaConf.structured(StyleEncoderConfig())

style_encoder = StyleEncoder(
    dim_in = style_encoder_cfg.dim_in, # 304
    style_dim = style_encoder_cfg.style_dim, # 128
    max_conv_dim = style_encoder_cfg.max_conv_dim, # 512
).to(device)



重みを読み込む場合は以下のようにする。
以下の "style_encoder.pth" は Notebook 02 で準備するデータセットを、Notebook 03 に示すコードで訓練したネットワークの state_dict である。こちらもリポジトリの他の構成要素（一部除く）と同じく、MIT License で配布される。


In [ ]:

# この場所に 作った重みを 置いておく
style_dict_path = "../weights/style_encoder.pth"

style_dict = torch.load(style_dict_path, map_location = device)
style_encoder.load_state_dict(style_dict, strict = True)


In [ ]:
import torch
import torchinfo

with torch.no_grad():    
    %time s_a = style_encoder(spec[:, 48:, :].unsqueeze(1))
print(s_a.shape)

torchinfo.summary(model = style_encoder, input_size = spec[:, 48:, :].unsqueeze(1).shape, depth = 4, 
                  col_names=["input_size", "output_size", "num_params"], device = device)




----

## Content encoder (ContentVec)

音声からの発話内容の抽出に使うのが、Quian et al. (2022) による ContentVec https://doi.org/10.48550/arXiv.2204.09224 である。
こちらも開発者による実装と重みが、以下のリポジトリにおいて MIT License で公開されている。

> Kaizhi Qian, Yang Zhang, Heting Gao, Junrui Ni, Cheng-I Lai, David Cox, Mark Hasegawa-Johnson, Shiyu Chang
> ContentVec: An Improved Self-Supervised Speech Representation by Disentangling Speakers

https://proceedings.mlr.press/v162/qian22b.html

https://arxiv.org/abs/2204.09224

https://github.com/auspicious3000/contentvec


ContentVec のネットワーク構造を定義するには、transformers パッケージの HubertModel を使う。
公式で配布されているコードを使うと、ONNX 化できないためである。詳しくは Notebook 04 で解説する。

> 重みは ContentVec の公式で配布している ContentVec_legacy の 500 class である。
> ただし若干アレンジする必要があるので、Notebook 04 に書いてある手順を参照して用意すること。
> 以下には最終成果物のモデルをロードする手順だけを示す。

* 入力は `torch.Size([n_batch, n_sample])` の 16 khz mono waveform だが、 `(1, 400)` （25 ms）以上ないとエラーになる。

* 隠れ層の最終次元を出力として使用し、そのサイズは `torch.Size([batch, ((length - 80) // 320), 768])` となる。

* HubertModel の生出力は feature last であるが、ほとんどの下流工程は time last になるので、転置して使うことに注意。

入力が 16000 Hz で 1/320 に間引くので **ContentVec の出力テンソルの hop は 20 ms である**。

* HarmoF0 pitch tracker は 10 ms hop なので、ちょうど端数以外は 2 倍のテンソルサイズとなる。



In [ ]:
from transformers import HubertConfig, HubertModel

CE = HubertModel(HubertConfig())

contentvec_path = proj_path / "contentvec_500_hubert.pth"

CE_dict = torch.load(str(contentvec_path), map_location = torch.device('cpu'))
CE.load_state_dict(CE_dict, strict = True)
CE.eval().to(device)


In [ ]:

with torch.no_grad():
    waveform, orig_sr = torchaudio.load('../wavs/p225_003.wav') # まずオリジナル周波数でロード
    wav16 = torchaudio.transforms.Resample(orig_freq = orig_sr, new_freq = 16000)(waveform).to(device)
    %time content = CE(wav16, output_hidden_states = True)

print(type(content))
print(len(content))
print(content.keys())
print(content["last_hidden_state"].shape) # テンソルを取り出すには ["last_hidden_state"] キーへのアクセスが必要なので注意


In [ ]:

torchinfo.summary(model = CE, input_size = wav16.shape, depth = 4, 
                  col_names=["input_size", "output_size", "num_params"], device = "cuda:0")


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm

fig, ax = plt.subplots()
im = ax.imshow(content["last_hidden_state"].cpu().squeeze().T, vmin = -1, vmax = 1)
fig.colorbar(im, ax = ax)
plt.tight_layout()
plt.show()


----

## ProsodyPredictor の定義

ContentVec の抽出した特徴量と、Style Encoder で抽出した話者スタイル $s_a$ をもとに、
F0 と energy の時間変化を予測するネットワークである。

原型となったネットワークは StyleTTS 2 に ProsodyPredictor として存在するが、
やはり ONNX 化が止まらないようにマイナーチェンジした `F0NPredictorAll` を作った。


In [ ]:
from pathlib import Path
from dataclasses import dataclass
import typing
from omegaconf import OmegaConf

from model.StyleTTS2.models import F0NPredictorAll

@dataclass
class PrododyPredictorConfig:
    style_dim: int = 128
    hidden_dim: int = 768
    n_layer: int = 3
    dropout: float = 0.2

prosody_predictor_cfg = OmegaConf.structured(PrododyPredictorConfig())

f0n_predictor = F0NPredictorAll(
    style_dim = prosody_predictor_cfg.style_dim,
    d_hid = prosody_predictor_cfg.hidden_dim,
    nlayers = prosody_predictor_cfg.n_layer,
    dropout = prosody_predictor_cfg.dropout,
).to(device)



In [ ]:

f0n_dict_path = "../weights/f0n_predictor.pth"

f0n_dict = torch.load(f0n_dict_path, map_location = device)
f0n_predictor.load_state_dict(f0n_dict, strict = True)


In [ ]:
# 新造した F0NPredictorAll は time last で入れる。返り値はそれぞれ feature 次元が潰れた 2D の time last だが、時間解像度が 2 倍

with torch.no_grad():
    %time pred_F0, pred_N = f0n_predictor(content["last_hidden_state"].transpose(2, 1), s_a)
    %time pred_F0, pred_N = f0n_predictor(content["last_hidden_state"].transpose(2, 1), s_a)

print(pred_F0.shape, pred_N.shape)


In [ ]:

torchinfo.summary(
    model = f0n_predictor, 
    input_size = (content["last_hidden_state"].transpose(2, 1).shape, s_a.shape), 
    depth = 3, 
    col_names = ["input_size", "output_size", "num_params"], 
    device = device,
)



`content` は 20 ms 間隔だが、`pred_F0`, `pred_N` は 10 ms 間隔である。

また上述のとおり HarmoF0 についても返り値は 10 ms 間隔だが、端数があるため 1 フレーム増えることがある。

なお次の decoder の入力系列長は、content を基準として 1/50 s = 20 ms 間隔で計算するシステムを採用している。




---


# VC Decoder の定義

StyleTTS 2 には iSTFTNet ベースのデコーダもあるが、スピードと音質を考えて、HiFi-GAN ベースだけをとりあえず考える。

> このデコーダの定義は RVC 等でも用いられている SourceModuleHnNSF に準拠しているようだが、
どうやら細部が異なる。多分 StyleTTS 2 で採用される以前に、元となった実装があるのだと思う。

```python
y_rec = decoder(
    en, # content 情報 torch.Size([1, 512, 445]) # 最終次元が長さ依存
    F0_real, # F0 torch.Size([1, 890]) # content 特徴量の長さの 2 倍
    real_norm, # Energy torch.Size([1, 890]) # content 特徴量の長さの 2 倍
    s, # acoustic style は固定サイズ torch.Size([1, 128])
)
```

上に書いたとおり decoder は基準長（content を使う）が 1/50 s = 20 ms ピッチである。
これをそのまま ConvTranspose で伸長して 24000 Hz 音声にしようとすると、24000/50 = 480 倍となる。

* なお内部定義的に、 `upsample_rates` の積和のさらに 2 倍が実伸長率になるので注意。

* StyleTTS 2 は `upsample_rates = [5, 5, 3, 2] × 2` だったので、そのままだと 300 倍になってしまう。

* なので ClassicVC ではとりあえず `upsample_rates = [10, 4, 3, 2] × 2` で 480 倍に変更した。

本当は音声を 48k 化したいが手が足りていない。 `upsample_rates = [10, 4, 3, 2, 2] × 2` で 960 倍にするのだと思うが、まだ安定して訓練できない。

> 一応アイデアとしては既存の [10, 4, 3, 2] アップサンプル部分までのネットワーク層について、24k 用に公開している重みを移植してその部分は凍結し、
新しく足した最後の 2 倍のアップサンプルだけを、話者数は少なくてもいいので高音質の音声データセットで訓練できるだろう。
暇な人はやってみて欲しい。



In [ ]:
from pathlib import Path
from dataclasses import dataclass
import typing
from omegaconf import OmegaConf
import math

from model.StyleTTS2.hifigan import Decoder

upsample_rate_list = [10, 4, 3, 2]

@dataclass
class DecoderConfig:
    sampling_rate: int = 24000
    dim_in: int = 768
    style_dim: int = 128
    upsample_rate_list: list = tuple(upsample_rate_list)
    upsample_kernel_list: list = tuple([i*2 for i in upsample_rate_list])
    upsample_total: int = math.prod(upsample_rate_list)*2
    upsample_initial_channel: int = 512
    harmonic_num: int = 8

decoder_cfg = OmegaConf.structured(DecoderConfig())

decoder = Decoder(
    sampling_rate = decoder_cfg.sampling_rate,
    dim_in = decoder_cfg.dim_in,
    style_dim = decoder_cfg.style_dim,
    resblock_kernel_sizes = [3, 7, 11], # ここは大多数のモデルで同じ設定値を採用している
    resblock_dilation_sizes = [[1, 3, 5], [1, 3, 5], [1, 3, 5]], # ここは大多数のモデルで同じ設定値を採用している
    upsample_rates = decoder_cfg.upsample_rate_list,
    upsample_initial_channel = decoder_cfg.upsample_initial_channel,
    upsample_kernel_sizes = decoder_cfg.upsample_kernel_list,
    harmonic_num = decoder_cfg.harmonic_num,
).to(device)



実は注意が必要で、
`F0_real`, `real_norm` は運用時は偶数のフレーム長でなければならない。
さもないと content が // 2 で短くなってしまうため、割り切れない長さになり、decoder 内の AdainResBlk1d でエラーが出る。


In [ ]:

decoder_dict_path = "../weights/decoder.pth"

decoder_dict = torch.load(decoder_dict_path, map_location = device)
decoder.load_state_dict(decoder_dict, strict = True)


In [ ]:

print(content["last_hidden_state"].shape)
print(pred_F0.shape, pred_N.shape)
print(s_a.shape)

with torch.no_grad():
    %time reconst = decoder(content["last_hidden_state"].transpose(2, 1), pred_F0, pred_N, s_a)
print(reconst.shape)


In [ ]:

torchinfo.summary(
    model = decoder, 
    input_size = (
        content["last_hidden_state"].transpose(2, 1).shape, pred_F0.shape, pred_N.shape, s_a.shape), 
    depth = 3, 
    col_names = ["input_size", "output_size", "num_params"], 
    device = device,
)


----

## 非リアルタイムで音声ファイルを声質変換する手順

音声から計算される特徴量の長さがフレーム単位で微妙に異なるので、
末尾から一定サイズを切り取る形で利用する。


In [ ]:
# ネットワークは上で定義した既存インスタンスを使っているので注意

def pred_f0_len(length):
    return length // 160 + 1

def VC(
    source_wav,
    target_wav = None,
    device = "cuda:0",
    reconst: bool = False,
    pred_prosody: bool = True,
    pitch: float = 0.0,
    return_originals: bool = False, # ソースおよびターゲットの waveform とサンプリング周波数も返す
    debug: bool = False,
):
    wav_s, s_sr = torchaudio.load(source_wav) # まずオリジナル周波数でロード
    w24_s = torchaudio.transforms.Resample(orig_freq = s_sr, new_freq = 24000)(wav_s)
    w16_s = torchaudio.transforms.Resample(orig_freq = s_sr, new_freq = 16000)(wav_s)
    seq_len = pred_f0_len(w16_s.size(-1)) # HarmoF0 が返すであろう系列長を計算しておく
    if debug:
        print("(input):", wav_s.shape, s_sr, w16_s.shape, seq_len)
    
    if target_wav is None:
        reconst = True
        if debug:
            print("'target_wav' is not specified. Reconstruction task will run.")
    else:
        wav_t, t_sr = torchaudio.load(target_wav) # まずオリジナル周波数でロード
        w24_t = torchaudio.transforms.Resample(orig_freq = t_sr, new_freq = 24000)(wav_t)
        w16_t = torchaudio.transforms.Resample(orig_freq = t_sr, new_freq = 16000)(wav_t)

    if reconst:
        with torch.no_grad():
            pitch_s, energy_s, act_s, spec_s = harmof0_tracker.to(device)(w16_s.to(device))
            acoustic_style_s = style_encoder.to(device)(spec_s[:, 48:, :].unsqueeze(1))
            prosodic_style_s = acoustic_style_s
            content_s = CE.to(device)(w16_s.to(device))["last_hidden_state"].transpose(2, 1)
            if debug:
                print("(reconst is True):", acoustic_style_s.shape, prosodic_style_s.shape, content_s.shape)
            if pred_prosody:
                pred_F0_s, pred_N_s = f0n_predictor.to(device)(content_s, prosodic_style_s) 
                if debug:
                    print("(pred_prosody is True):", pred_F0_s.shape, pred_N_s.shape) 
                result = decoder.to(device)(
                    content_s, 
                    pred_F0_s, 
                    pred_N_s, 
                    acoustic_style_s,
                )
            else:
                if debug:
                    print("(pred_prosody is False):", content_s.shape, pitch_s.shape) 
                result = decoder.to(device)(
                    content_s, 
                    pitch_s[:, -content_s.shape[-1]*2:] * 2**(pitch/12), # 予測 pitch の方が長い（正確には content が短い）
                    energy_s[:, -content_s.shape[-1]*2:], 
                    acoustic_style_s,
                )
    else:
        with torch.no_grad():
            pitch_s, energy_s, act_s, spec_s = harmof0_tracker.to(device)(w16_s.to(device))
            pitch_t, energy_t, act_t, spec_t = harmof0_tracker.to(device)(w16_t.to(device))
            acoustic_style_t = style_encoder.to(device)(spec_t[:, 48:, :].unsqueeze(1))
            prosodic_style_t = acoustic_style_t
            content_s = CE.to(device)(w16_s.to(device))["last_hidden_state"].transpose(2, 1)
            if debug:
                print("(reconst is False):", acoustic_style_t.shape, prosodic_style_t.shape, content_s.shape) 
            if pred_prosody:
                pred_F0_st, pred_N_st = f0n_predictor.to(device)(content_s, prosodic_style_t)
                if debug:
                    print("(pred_prosody is True):", pred_F0_st.shape, pred_N_st.shape) 
                result = decoder.to(device)(
                    content_s, 
                    pred_F0_st * 2**(pitch/12), 
                    pred_N_st, 
                    acoustic_style_t,
                )
            else:
                pitch_s, energy_s, act_s, _ = harmof0_tracker.to(device)(w16_s.to(device))
                if debug:
                    print("(pred_prosody is False):", content_s.shape, pitch_s.shape) 
                result = decoder.to(device)(
                    content_s, 
                    pitch_s[:, -content_s.shape[-1]*2:] * 2**(pitch/12), 
                    energy_s[:, -content_s.shape[-1]*2:], 
                    acoustic_style_t,
                )
        
    if result.max() > 1 or result.min() < -1:
        print(f"Range of the synthesized waveform ({result.min()}, {result.max()}) exceeds [-1, 1] and clamped.")
        result = torch.clamp(result, -1, 1)

    if return_originals:
        if reconst:
            return result, w24_s, s_sr
        else:
            return result, w24_s, s_sr, w24_t, t_sr
    else:
        return result


In [ ]:

source_link = '../wavs/p225_003.wav'
target_link = '../wavs/p227_001.wav'

# VC を実行。ピッチを変換先スタイルに合わせる
wav_vc, wav_source, sr_source, wav_target, sr_target = VC(
    source_link, 
    target_link, 
    reconst = False, 
    pred_prosody = True, 
    return_originals = True, 
    debug = True,
)

# target を指定しない → 再構成タスク
wav_vc, wav_source, sr_source = VC(
    source_link, 
    reconst = False, 
    pred_prosody = True, 
    return_originals = True, 
    debug = True,
)

# target はあるが、再構成を指定。またピッチを変換先ではなくソース音声の話者スタイルに合わせる
wav_vc, wav_source, sr_source = VC(
    source_link, 
    target_link, 
    reconst = True, 
    pred_prosody = False, 
    return_originals = True, 
    debug = True,
)

# オリジナルを返さない。ピッチを元音声から +6 半音上げる
wav_vc = VC(
    source_link, 
    target_link, 
    reconst = False, 
    pred_prosody = False, 
    pitch = 6.0,
    return_originals = False, 
    debug = True,
)




以下は変換前後の音声を Notebook 上で再生するための便利関数



In [ ]:
import IPython.display as ipd

def display_audio_grid(
    wavs,
    titles,
    sr = decoder_cfg.sampling_rate,
):
    nrow = len(wavs)

    # 音声データとタイトルのリスト
    audio_html = ""
    for wav_row, title_row in zip(wavs, titles):
        wav_row = [w.detach().cpu().squeeze().numpy() for w in wav_row]
        audio_html = audio_html + "<table><tr>{}</tr></table>".format(
            "".join([f"<td><b>{title}</b><br>{ipd.Audio(audio, rate = sr, normalize = True)._repr_html_()}</td>" 
            for audio, title in zip(wav_row, title_row)])
        )
    # 音声データとタイトルを横に並べて表示
    display(ipd.HTML(audio_html))


In [ ]:
source_link = '../wavs/p225_003.wav'
target_link = '../wavs/p227_001.wav'

# VC を実行。ピッチを変換先スタイルに合わせる
wav_vc, w24_source, sr_source, w24_target, sr_target = VC(
    source_link, 
    target_link, 
    reconst = False, 
    pred_prosody = True, 
    return_originals = True, 
)

# VC を実行。ピッチを変換先スタイルに合わせる
wav_vc_reverse = VC(
    target_link, 
    source_link, 
    reconst = False, 
    pred_prosody = True, 
    return_originals = False, 
)

recon_source = VC(
    source_link, 
    reconst = True, 
    pred_prosody = False, 
    return_originals = False, 
)

recon_target = VC(
    target_link, 
    reconst = True, 
    pred_prosody = False, 
    return_originals = False, 
)

# 左上：ソース音声の GT、右下：ターゲット音声の GT

display_audio_grid(
    [
        [w24_source, wav_vc_reverse],
        [recon_source, recon_target],
        [wav_vc, w24_target],
    ],
    [
        ["225", "227 to 225"],
        ["225 recon", "227 recon"],
        ["225 to 227", "227"],
    ]
)


ここで投入したのはいずれも VCTK コーパスから抜粋した音声クリップであり、
元のモデルの訓練には用いられていないデータである。

喋っていない部分も微妙に声っぽいものが入ってしまうが、概ね未知話者間での声質変換（zero-shot voice conversion）が可能なことが分かるだろう。


